<a href="https://colab.research.google.com/github/dharsha454/Test-documents/blob/main/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install pdfplumber library to extract text from PDF

In [ ]:
pip install pdfplumber

Import pdfplumber, regex

In [2]:
import pdfplumber
import re

In [3]:
import spacy

In [4]:
nlp = spacy.load("en_core_web_sm")

Keys to be extracted

In [5]:
keys=['Temperature','Height','Weight','Blood Pressure']

Load File 

In [6]:
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://github.com/dharsha454/Test-documents/raw/main/Test.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   139  100   139    0     0    785      0 --:--:-- --:--:-- --:--:--   785
100  108k  100  108k    0     0   301k      0 --:--:-- --:--:-- --:--:-- 4221k


Text Extraction

In [7]:
with pdfplumber.open(r'Test.pdf') as pdf:
    curr = pdf.pages[0]
    text = (curr.extract_text())

In [ ]:
print(text)

In [9]:
result={}
j=''

Extracting key value pairs using regex

In [10]:

for i in keys:
  if i=='Blood Pressure':
    j = re.search(r'{}\w*:\d*\/\d*'.format(i),text).group()

  elif re.search(r'{}\w*:\d*'.format(i),text):
    j = re.search(r'{}\w*:\d*'.format(i),text).group()
   
  elif re.search(r'{}\w*:\d*'.format(i),text):
    j = re.search(r'{}\w*:\d*'.format(i),text).group()

  j= j.replace('\n','')
  j = j.replace(':','') 
  result[i] = j.replace(i,'')

In [11]:
result

{'Blood Pressure': '130/80',
 'Height': '175',
 'Temperature': '98',
 'Weight': '170'}

Exporting to json

In [12]:
import json
with open('output.json', 'w') as f:
    json.dump(result,f)

In [13]:
test_json = json.load(open('output.json'))
test_json

{'Blood Pressure': '130/80',
 'Height': '175',
 'Temperature': '98',
 'Weight': '170'}

In [14]:
doc = nlp(text)

In [15]:
from spacy import displacy

In [ ]:
displacy.render(nlp(doc.text), style='ent', jupyter=True)

In [19]:
train = [

("Temperature is 95", {"entities": [(0, 10, "PARAM"), (15, 16, 'VALUE')]}),
("He has a body temperature of 100", {"entities": [(14, 24, "PARAM"), (29, 31, 'VALUE')]}),
("she checked her body temperature to be 98.2", {"entities": [(21, 31, "PARAM"), (39, 42, 'VALUE')]}),
("Temperature is 97.5", {"entities": [(0, 10, "PARAM"), (15, 18, 'VALUE')]}),
("The Blood Pressure of Ravi is 120/80", {"entities": [(4, 17, "PARAM"), (30, 36, 'VALUE')]}),
("Sheila has a Blood Pressure of 135/90", {"entities": [(13, 26, "PARAM"), (31, 37, 'VALUE')]}),
("Blood Pressure of 135/95 is not ideal", {"entities": [(0, 13, "PARAM"), (18, 23, 'VALUE')]}),
("Height of Ravi is 175", {"entities": [(0, 5, "PARAM"), (18, 20 , 'VALUE')]}),
("Height is 155", {"entities": [(0, 5, "PARAM"), (10, 12, 'VALUE')]}),
("The Height of his wife is 168", {"entities": [(4, 10, "PARAM"), (26, 29, 'VALUE')]}),
("Ram checked his Weight to be 85", {"entities": [(16, 22, "PARAM"), (29, 30, 'VALUE')]}),
("Weight of Chris is 90.8", {"entities": [(0, 5, "PARAM"), (19, 22, 'VALUE')]}),
("Average Weight of all students in the class is 52.5", {"entities": [(8, 13, "PARAM"), (47, 50, 'VALUE')]})

]

In [20]:
nlp.pipe_names

['tagger', 'parser', 'ner']

In [21]:
ner = nlp.get_pipe("ner")

In [22]:
for _, annotations in train:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

In [23]:
disable_pipes = [pipe for pipe in nlp.pipe_names if pipe!='ner']

In [24]:
import random
from spacy.util import minibatch,compounding
from pathlib import Path

In [ ]:
with nlp.disable_pipes(*disable_pipes):
  optimizer = nlp.resume_training()
  for iteration in range(100):
    random.shuffle(train)
    losses = {}
    batches = minibatch(train, size = compounding(1.0, 4.0, 1.001))
    for batch in batches:
      text, annotation = zip(*batch)
      nlp.update(
          text,
          annotation,
          drop=0.5,
          losses=losses,
          sgd=optimizer
      )
      print("Losses", losses)





In [28]:
for text, _ in train:
  doc = nlp(text)
  print('Entities', [(ent.text, ent.label_) for ent in doc.ents])


Entities [('98.2', 'CARDINAL')]
Entities [('Temperature', 'FAC'), ('97.5', 'FAC')]
Entities []
Entities [('Height', 'FAC'), ('155', 'CARDINAL')]
Entities [('Weight', 'FAC'), ('90.8', 'CARDINAL')]
Entities [('Temperature', 'FAC'), ('95', 'FAC')]
Entities [('100', 'CARDINAL')]
Entities []
Entities [('Weight', 'PARAM'), ('52.5', 'VALUE')]
Entities [('Height', 'PARAM'), ('168', 'VALUE')]
Entities [('Height', 'FAC')]
Entities [('135/90', 'VALUE')]
Entities [('Weight', 'PARAM'), ('85', 'VALUE')]


In [ ]:
from spacy import displacy
doc = nlp("My Height is 158")

for ent in doc.ents:
   print(ent.text, ent.label)

displacy.render(nlp(doc.text), style='ent', jupyter=True)

In [ ]:
from spacy import displacy
doc = nlp("Weight is 78")

for ent in doc.ents:
  print(ent.text, ent.label)

displacy.render(nlp(doc.text), style='ent', jupyter=True)